# Image Recommendation System via Collaborative Filtering

# ***Please read the instructions very carefully***

1.   Assignment must be implemented in Python 3 only.
2.   You are allowed to use libraries for data preprocessing (numpy, pandas, nltk etc) and for evaluation metrics, data visualization (matplotlib etc.).
3.   You will be evaluated not just on the overall performance of the model and also on the experimentation with hyper parameters, data prepossessing techniques etc.
4.   ⚠️ The Assignment will be evaluated automatically. Please adhere to taking proper inputs from `config.csv` file. You can change your `config.csv` file to experiment with your code. But at the end, make sure that your outputs are corresponding to input values in `config.csv`
5.   Strict plagiarism checking will be done. An F will be awarded for plagiarism.

## About the Dataset
Behance is a community art website where users showcase and discover creative work. Each user is able to “appreciate” (equivalent to a “like” on Instagram or a “react” on Facebook) an image, indicating that they like the image. It is in the website’s best interests to show users pictures that they would like, to keep them engaged for longer. For this question, given a set of pictures that a user has already appreciated, you have to show them a new picture that they would like based on what similar users appreciated.


<br><br>
**The dataset has information of 1 million appreciates of 63,497 users on 178,788 items. The file Behance appreciate 1M has a triplet in each line in the form of (user id, item id, unix timestamp).**

**Task: Take the inputs from the config.csv file and output the recommendations for a particular person**
- Collaborative Filtering is a way to predict items to the user based on the the
user’s history and the history of similar users. The similarity between users can be quantified by the number of images that both the users appreciated.
- The images appreciated by a similar user would be the most suitable images to show a user. Since we can find the similarity between any two users, we would be able to find the “nearest” neighbours of any user, allowing us to use a KNN-based algorithm to recommend new images to a user.
- Since people do not like seeing pictures that they have seen already. Make sure that you do not recommend pictures that a user has appreciated already.
- Output the final response will be saved in the file named ```config['output_file']```.


**Output file format:**
Populate the output file with images that the user has not seen of the k most
similar users, in descending order of their similarity. Each line in the output
file should be a duplet in the form of (item id, user id), where the user id is the
id of the kth similar user. The order of the images corresponding to the same
similar user would not matter. The output file would look something like this:
```
item_id_1_of_1st_similar_user 1st_most_similar_user_id
item_id_2_of_1st_similar_user 1st_most_similar_user_id
item_id_3_of_1st_similar_user 1st_most_similar_user_id
...
item_id_1_of_2nd_similar_user 2nd_most_similar_user_id
item_id_2_of_2nd_similar_user 2nd_most_similar_user_id
item_id_3_of_2nd_similar_user 2nd_most_similar_user_id
...
item_id_1_of_kth_similar_user kth_most_similar_user_id
item_id_2_of_kth_similar_user kth_most_similar_user_id
item_id_3_of_kth_similar_user kth_most_similar_user_id
```

The dataset was extracted using Behance’s API as a part of the paper
“Vista: A visually, socially, and temporally-aware model for artistic
recommendation, RecSys, 2016”. Check out this [Google Drive folder](https://drive.google.com/drive/folders/0B9Ck8jw-TZUEc3NlMjVXdDlPU1k?resourcekey=0-6_8ykn0o4fLc5fuTEm91xA) for
more information about the dataset.


Have fun! The users are waiting to see new pictures!

### Import necessary libraries

In [11]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from numpy import array
from scipy.linalg import svd
from sklearn.decomposition import TruncatedSVD
from numpy import dot
from numpy.linalg import norm

In [12]:
# Config Generation Sample Code.
# ⚠️ Only for experimentation on your side.
# ⚠️ Should be commented during submission.

# df = pd.DataFrame(data=[{'id':276633,
#                   'k':5,
#                   'dataset_file':'./Behance_appreciate_1M',
#                   'output_file':'./output.txt'}])
# df.to_csv('config.csv')

In [13]:
config = pd.read_csv('config.csv').iloc[0]

In [14]:
config

In [15]:
user = config['id']
k_value = config['k']

### Read the Data

In [16]:
with open(config['dataset_file'], 'r') as inFile:
    appreciate_data = inFile.readlines()

### Initialize a dictionary to store the item_ids that a user likes

### Go through each line of the input file and construct the user_likes dictionary

In [17]:
user_likes = dict()
users={}
items={}
u_ind=0
i_ind=0
user_row=[]
item_col=[]
matrix_val=[]
knn=[]

In [18]:
for line in appreciate_data:
    line = line.strip()
    
    user_id = int(line.split()[0])
    item_id = int(line.split()[1])

    if(user_id not in users):
        users[user_id]=u_ind
        u_ind+=1

    if(item_id not in items):
        items[item_id]=i_ind
        i_ind+=1
    
    user_row.append(users[user_id])
    item_col.append(items[item_id])
    matrix_val.append(1)
    

    if user_id not in user_likes:
        user_likes[user_id] = list()
    
    user_likes[user_id].append(item_id)

In [19]:
# print(len(users))
# print(len(items))

# print(len(user_row))
# print(len(item_col))
# print(len(matrix_val))
original_user_images=user_likes[user]
user_row=np.array(user_row)
item_col=np.array(item_col)
matrix_val=np.array(matrix_val)

csr = csr_matrix((matrix_val, (user_row, item_col)), shape=(len(users), len(items)))
# print(csr)
Singular_Value_Decomposition = TruncatedSVD(n_components=1000)

my_matrix =Singular_Value_Decomposition.fit_transform(csr)







In [ ]:
# print(my_matrix)
# print(my_matrix[0])
# print(users[user])

def cosine_distance(v1,v2):
    # v1=np.array(v1)
    # v2=np.array(v2)
    result = 1-(dot(v1, v2)/(norm(v1)*norm(v2)))
    return result

distances={}
user_index=users[user]
for this_user in range(len(my_matrix)):
    if(this_user!=user_index):
        distances[this_user]=cosine_distance(my_matrix[this_user],my_matrix[user_index])

sorted_distances = sorted(distances.items(),key=lambda x:x[1])
sorted_distances=sorted_distances[:k_value]
for item in sorted_distances:
  knn.append(list(users.keys())[list(users.values()).index(item[0])])
# print(knn)






### Use KNN after Collaborative Filtering to find nearest neighbors

In [ ]:
# your code here
# your code here
def neighbors(user,k_value):
    """ returns an iterable object (like list or generator) """
    pass

### Open the output file to write all the lines to the file

In [ ]:
outFile = open(config['output_file'], 'w')

for n_user in knn:
    # user_id = list(user_likes.keys())[n_user]
    user_id=n_user
    # print("userid is ",user_id)
    for item_id in user_likes[user_id]:
        if(item_id not in original_user_images):
            outFile.write(str(item_id) + ' ' + str(user_id) + '\n')

outFile.close()

userid is  1973004
userid is  1494939
userid is  1480429
userid is  2452817
userid is  2539765
